Hinweis vorab: Es wird eine Bibliothek benötigt. Diese installieren Sie mit pip install py_dempster_shafer

# Dempster Shafer Übungsbeispiel


Ein  Kriminalfall  soll  mit  Hilfe  der  Evidenzen  und  Dempsters  Regel  ge-löst  werden.  Hier  ist  die  nötige  Information:  Einbruch  beim  Nachbarn.  Wer  war  der  Täter?  Es  gibt  eine  Menge  von  Verdachtspersonen  mit  folgenden Eigenschaften: 

###### Zeugenaussage 1

„Ich glaube der Täter war dunkelhaarig, ziemlich sicher ...“

Dies trifft auf Tobias, Ralf und Sabine (t,r,s) zu. Diese Menge ist also nicht weiter differenzierbar aber insgesamt ist für diese nun eine Aussage möglich.

In [25]:
from pyds import MassFunction

omega = 'trsuef'

Das Basismaß wird in der Bibliothek als "MassFunction" implementiert. Die Evidenz wird nun für dieses Beispiel mit 0.88 festgelegt.

In [26]:
m1 = MassFunction({'trs':0.88, omega:0.12})
print('m1 =', m1)

m1 = {{'r', 's', 't'}:0.88; {'r', 'u', 's', 't', 'e', 'f'}:0.12}


###### Zeugenaussage 2

 „Zwei  kämen  in  Frage:  einer  sehr  groß,  bestimmt  zwischen  1,80 und 2 m – und der andere eher klein, kleiner als 1,70 m.“ 


Dies trifft auf Tobias, Sabine und Frank zu (groß) und auch auch Ulrike und Ralf (klein). Die Evidenzen werden jeweils mit 0.45 festgelegt, so dass noch 0.1 für Omega übrig bleibt.

In [27]:
m2 = MassFunction({'tsf':0.45,'ur':0.45,omega:0.1})
print('m2 =', m2)

m2 = {{'f', 's', 't'}:0.45; {'r', 'u'}:0.45; {'r', 'u', 's', 't', 'e', 'f'}:0.1}


In [28]:
m12 = m1.combine_conjunctive(m2) # dies ist Demptsters Regel wie in der Vorlesung vorgestellt
print('m12 = ',m12)

m12 =  {{'s', 't'}:0.396; {'r'}:0.396; {'r', 's', 't'}:0.08800000000000001; {'s', 't', 'f'}:0.054; {'r', 'u'}:0.054; {'r', 'u', 's', 't', 'e', 'f'}:0.012}


###### Zeugenaussage 3
„Ich würde die Person mal auf über 40 schätzen ...“ 

In [29]:
m3 = MassFunction({'uer':0.65,omega:0.35})
print('m3 =', m3)

m3 = {{'r', 'u', 'e'}:0.65; {'r', 'u', 's', 't', 'e', 'f'}:0.35}


Nun kombinieren wir auch diese mit dem Ergebnis der ersten Verknüpfung von Zeugenaussage 1 und 2.

In [30]:
m123 = m12 & m3
print('m123 = ', m123)

m123 =  {{'r'}:0.640565371024735; {'s', 't'}:0.19590106007067137; {'r', 'u'}:0.07632508833922261; {'r', 's', 't'}:0.04353356890459364; {'f', 's', 't'}:0.026713780918727916; {'r', 'u', 'e'}:0.011024734982332156; {'r', 'u', 's', 't', 'e', 'f'}:0.005936395759717314}


Wie plausibel ist es nun, dass Ralf der Täter ist?

In [33]:
m123.pl({'r'})

0.7773851590106007

Eine kleine Ergänzung zur Bestimmung des Konflikts: 

In [49]:
m12.conflict(m3)

0.34601764946474434

Der Wert entspricht nicht dem 'K', welches in der Vorlesung eingeführt wurde. K ist hier 0,2925. Für die Normalisierung mussten alle verbleibenden Elemente des Maßes mit 1/(1-K) = 1,4134 multipliziert werden. Um diese mit Hilfe dieser Bibliothek nachzustellen ist zu beachten, dass diese den natürlichen Logarithmus des Normierungsfaktors als Konflikt bezeichnet.

In [52]:
import math
N = math.exp(m12.conflict(m3))
K = (1-N)/-N # entspricht N = 1/(1-K)
print(K)

0.29250000000000004


Darüber hinaus kann noch der Belief bestimmt werden. 

In [54]:
print('B(Ralf) =', m123.bel('r'))
print('B(Tobias) =',m123.bel('t'))
print('B(Frank) =',m123.bel('f'))

B(Ralf) = 0.640565371024735
B(Tobias) = 0.0
B(Frank) = 0.0


In [56]:
print('B(Es war ein Mann)  = B({t,r,f}) = ',m123.bel('trf'))
print('B(Es war eine Frau) = B({s,u,e}) = ',m123.bel('sue'))

B(Es war ein Mann)  = B({t,r,f}) =  0.640565371024735
B(Es war eine Frau) = B({s,u,e}) =  0.0


In [58]:
print('Pl(Es war ein Mann)  = Pl({t,r,f}) = ',m123.pl('trf'))
print('Pl(Es war eine Frau) = Pl({s,u,e}) = ',m123.pl('sue'))

Pl(Es war ein Mann)  = Pl({t,r,f}) =  1.0
Pl(Es war eine Frau) = Pl({s,u,e}) =  0.359434628975265


Der Zweifel(x) ist nicht direkt implementiert, kann aber ergänzt werden durch die Berechnung des Beliefs an Omega ohne x. 

In [60]:
m123.bel(omega.replace('r',""))

0.2226148409893993